## Libraries
This notebook explains how to retrieve data for and edit private and public ADS libraries.

The base URL for library queries is
```
https://api.adsabs.harvard.edu/v1/biblib/<endpoint>
```
where the allowed `<endpoint>` values are:
* `/libraries`: create or view all libraries for a given user; GET or POST methods allowed
* `/libraries/<library_id>`: view a specific library; GET method is allowed
* `/documents/<library_id>`: add and remove records from a specific library, update the metadata for a specific library, or delete an entire library; POST, PUT, and DELETE methods are allowed
* `/permissions/<library_id>`: view or modify the permissions for a specific library; GET and POST methods are allowed
* `/transfer/<library_id>`: transfer ownership of a library to another ADS user; GET and POST methods are allowed

These endpoints are discussed further below, with examples.

Note that the complete curl request should also include your personal API access token, as described in the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md).

The API examples in this notebook generally assume you're using `curl` statements from the shell command line--you'll see `%%bash` at the top of a cell if its contents should be run in the shell instead of in Python. Guidelines for translating bash `curl` statements into Python can be found in the [converting curl to python notebook](Converting_curl_to_python.ipynb). If you prefer, you can use the unofficial Python package instead; see the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) for more information.

### Permissions
  * **owner**: Owns the library. Can read libraries, add/remove bibcodes, delete the library, add/remove permissions, transfer ownership of the library.
  * **admin**: Can read libraries, add/remove bibcodes, add/remove permissions.
  * **write**: Can read libraries, add/remove bibcdes.
  * **read**: Can read libraries.

Any other user that does not have a permission to the library they try to visit, can only see it if it is public. If the user has no permissions, they cannot view this library.

### `/libraries`
Create a library or view all libraries for a given user, depending on the method used.

#### GET method
View all libraries for the user associated with the given API token.

*Return data*:
  * name: (string) name of the library
  * id: (string) ID of the library
  * description: (string) description of the library
  * num_documents: (integer) number of documents in the library
  * date_created: (string) ISO date library was created
  * date_last_modified: (string) ISO date library was last modified
  * permission: (string) permission type, can be: 'read', 'write', 'admin', or 'owner'
  * public: (boolean) true means it is a public library
  * num_users: (integer) number of users with permissions to this library
  * owner: (string) ID of the user who created the library

In [1]:
%%bash
token="your-token-here"
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/biblib/libraries

{"libraries": [{"num_documents": 163, "date_last_modified": "2018-06-28T15:30:24.764691", "permission": "owner", "description": "My ADS library", "public": false, "num_users": 1, "owner": "test.user-api", "date_created": "2018-06-28T15:30:24.764684", "id": "ZX9Mgv67QSyS6GfuYLjB7w", "name": "ADS"}, {"num_documents": 100, "date_last_modified": "2018-06-28T15:32:10.641190", "permission": "owner", "description": "My ADS library", "public": false, "num_users": 1, "owner": "test.user-api", "date_created": "2018-06-28T15:31:12.741894", "id": "HNHFavGNS3izENGEbGDJCA", "name": "WMAP"}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   595  100   595    0     0    595      0  0:00:01 --:--:--  0:00:01  1882


#### POST method
Create a library for the requesting user. Returns the metadata of the library created.

In addition to the usual token header and request URL, you must specify that this is a POST request using `-X POST`. The POST payload must also be passed using the `-d` flag and should be in JSON format. The payload should include the following keys:
  * name: (string) optional; name of the library (must be unique for that user). The default is `Untitled Library`, with an incremented integer starting with 1 appended
  * description: (string) optional; description of the library. The default is "My ADS library"
  * public: (boolean) optional; set to true to allow public access. Note: the Boolean value is case senstitive, and must be entered in lowercase. The default is false, or a private library
  * bibcode: (list) optional; list of bibcodes to add. The default is an empty library
  
To indicate that the payload is in JSON format, you must include a header specifying the format: `-H "Content-Type: application/json"`.

The metadata of the library you just created will be returned after the request:
  * name: (string) name of the library
  * id: (string) ID of the library
  * description: (string) description of the library

In [2]:
%%bash
token="your-token-here"
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries \
    -X POST \
    -d '{"name":"API library", "description":"This library is created via API", "public":true, "bibcode":["1975CMaPh..43..199H","1973PhRvD...7.2333B"]}'


{"id": "hHGU1Ef-TpacAhicI3J8kQ", "bibcode": ["1975CMaPh..43..199H", "1973PhRvD...7.2333B"], "description": "This library is created via API", "name": "API library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   308  100   165  100   143    165    143  0:00:01 --:--:--  0:00:01   980


### /libraries/&lt;library_id&gt;
View a specific library. The `library_id` (string) is the unique identifier of the library, given via the `/libraries` GET method detailed above.

To edit or delete a library, see the `/document/&lt;library_id&gt;` endpoint section below.

#### GET
Obtains all the documents in a given library. 

*Return data*:
  * documents: (list) a list containing the bibcodes in the library
  * solr: (dict) the response from the solr bigquery endpoint
  * metadata: (dict) Contains the following:
        * name: (string) name of the library
        * id: (string) ID of the library
        * description: (string) description of the library
        * num_documents: (integer) number of documents in the library
        * date_created: (string) ISO date library was created
        * date_last_modified: (string) ISO date library was last modified
        * permission: (sting) Permission type, can be 'read', 'write', 'admin', or 'owner'
        * public: (boolean) true means it is public
        * num_users: (integer) number of users with permissions to this library
        * owner: (string) identifier of the user who created the library
  * updates: (dict) Contains the following
        * num_updated: (integer) number of documents modified based on the response from solr
        * duplicates_removed: (integer) number of files removed because they are duplications
        * update_list: (list, dict) list of dictionaries such that a single element described the original bibcode (key) and the updated bibcode now being stored (item)

The following type of user can read a library:
  - owner
  - admin
  - write
  - read

In [3]:
%%bash
token="your-token-here"
# get the data for the library created above
# note: add | python -m json.tool to the end of the request to print the response in a more readable format
curl -H "Authorization: Bearer $token" https://api.adsabs.harvard.edu/v1/biblib/libraries/hHGU1Ef-TpacAhicI3J8kQ | python -m json.tool

{
    "documents": [
        "1975CMaPh..43..199H",
        "1973PhRvD...7.2333B"
    ],
    "metadata": {
        "date_created": "2018-06-28T17:50:14.641552",
        "date_last_modified": "2018-06-28T17:50:14.641558",
        "description": "This library is created via API",
        "id": "hHGU1Ef-TpacAhicI3J8kQ",
        "name": "API library",
        "num_documents": 2,
        "num_users": 1,
        "owner": "test.user-api",
        "permission": "owner",
        "public": true
    },
    "solr": {
        "response": {
            "docs": [
                {
                    "bibcode": "1975CMaPh..43..199H"
                },
                {
                    "bibcode": "1973PhRvD...7.2333B"
                }
            ],
            "numFound": 2,
            "start": 0
        },
        "responseHeader": {
            "QTime": 22,
            "params": {
                "fl": "bibcode,alternate_bibcode",
                "fq": "{!bitset}",
                "q": "*:*",

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   780  100   780    0     0    780      0  0:00:01 --:--:--  0:00:01  1115


### /libraries/operations/&lt;library_id&gt;
Perform set operations on one or more libraries. The `library_id` (string) is the unique identifier of the primary library, as specified in the `/libraries` GET response. Depending on the operation, a secondary library ID may need to be passed in the post payload.

Function allowed: POST

#### POST
The following operations are supported:
* union: take the union of the primary and the secondary libraries. The result is saved to a new library
* intersection: take the intersection of the primary and the secondary libraries. The result is saved to a new library
* difference: take the difference between the primary and the secondary libraries. The result is saved to a new library
* copy: copy the contents of the primary library into the secondary library. The secondary library is not emptied first; use the empty operation on the secondary library first in order to create a duplicate of the primary library in the secondary library
* empty: empty the primary library of its contents (no secondary library ID is needed)

Specify that the request is a POST method by adding `-X POST` to your request. You must include a payload, passed via the `-d` flag, in JSON format. Add this header to your request to indicate your payload is in the appropriate format: `-H "Content-Type: application/json"`. The payload must include:
* action: (string, required) set operation to perform; allowed values are [`union`, `intersection`, `difference`, `copy`, `empty`]
* libraries: (list) list of secondary library IDs; multiple secondary libraries are allowed for [`union`, `intersection`, `difference`]; one secondary library is allowed for `copy`; no secondary libraries are allowed for `empty`
* name: (string, optional) name of the new library to be created for [`union`, `intersection`, `difference`]; the name must be unique for a given user; if no name is specified, the name used will be "Untitled" with a timestamp
* description: (string, optional) description of the new library to be created for [`union`, `intersection`, `difference`]; if no description is specified, the description used will include the set operation used and the primary and secondary library IDs
* public: (Boolean, optional) sets whether the new library created by [`union`, `intersection`, `difference`] is publicly viewable or not; the new library will be private unless otherwise specified

*Return data*:
* name: (string) Name of the library
* id: (string) ID of the library
* description: (string) Description of the library

*Permissions*:
The following type of user can perform set operations:
* owner
* admin
* write


In [4]:
%%bash
token="your-token-here"
# take the union of 3 libraries, using the default description and public settings
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/iRZhyhvaSWe0uWwcXKt77w \
    -X POST \
    -d '{"action": "union", "libraries": ["rdRs4g4AQ522k9TJ2umtVA", "s56fjycaRMKxw6tDEr31XQ"], "name": "New union library"}'

{"id": "BeUbSWXtTPCaRhRV3nec_w", "bibcode": ["2009A&A...502..515G", "2011ApJ...734...87R", "2010ApJ...710..248C", "2003ApJ...595..665X", "2011ApJ...731L..12S", "2010ApJ...721..505R", "2010ApJ...710L.156R", "2010ApJ...723.1255R", "2006ApJ...639L..51A", "2001AJ....122.2993S", "2010ASPC..423..355R", "2010A&A...518A..59G", "2010ApJ...721L..48K"], "description": "Union of 891661ca-1bda-4967-b4b9-6c1c5cab7bef with [u'rdRs4g4AQ522k9TJ2umtVA', u's56fjycaRMKxw6tDEr31XQ']", "name": "New union library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   613  100   498  100   115    498    115  0:00:01 --:--:--  0:00:01  2043


In [5]:
%%bash
token="your-token-here"
# take the intersection of 2 libraries, using the default public settings
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/BeUbSWXtTPCaRhRV3nec_w \
    -X POST \
    -d '{"action": "intersection", "libraries": ["rdRs4g4AQ522k9TJ2umtVA"], "name": "New intersection library", "description": "Sample intersection library"}'

{"id": "8V8DzNqyQuOyTX6RF9x7qQ", "bibcode": ["2001AJ....122.2993S", "2009A&A...502..515G", "2006ApJ...639L..51A", "2010ApJ...710..248C", "2003ApJ...595..665X", "2010A&A...518A..59G", "2011ApJ...731L..12S"], "description": "Sample intersection library", "name": "New intersection library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   438  100   289  100   149    289    149  0:00:01 --:--:--  0:00:01  2147


In [6]:
%%bash
token="your-token-here"
# take the difference of 2 libraries
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/BeUbSWXtTPCaRhRV3nec_w \
    -X POST \
    -d '{"action": "difference", "libraries": ["rdRs4g4AQ522k9TJ2umtVA"], "name": "New difference library", "description": "Sample difference library", "public": true}'

{"id": "9KBh8sVHQxaVkoPALtoXgg", "bibcode": ["2011ApJ...734...87R", "2010ApJ...723.1255R", "2010ApJ...710L.156R", "2010ASPC..423..355R", "2010ApJ...721L..48K", "2010ApJ...721..505R"], "description": "Sample difference library", "name": "New difference library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   421  100   262  100   159    262    159  0:00:01 --:--:--  0:00:01  2170


In [7]:
%%bash
token="your-token-here"
# empty a library of all of its bibcode contents
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/9KBh8sVHQxaVkoPALtoXgg \
    -X POST \
    -d '{"action": "empty"}'

{"public": true, "bibcode": [], "name": "New difference library", "description": "Sample difference library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   129  100   110  100    19    110     19  0:00:01 --:--:--  0:00:01   689


In [8]:
%%bash
token="your-token-here"
# copy the bibcode contents from the primary library to the secondary library (this will not empty the library first)
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/BeUbSWXtTPCaRhRV3nec_w \
    -X POST \
    -d '{"action": "copy", "libraries": ["9KBh8sVHQxaVkoPALtoXgg"]}'

{"public": true, "bibcode": ["2009A&A...502..515G", "2010ApJ...710L.156R", "2010ApJ...710..248C", "2003ApJ...595..665X", "2011ApJ...731L..12S", "2010ApJ...721..505R", "2011ApJ...734...87R", "2010ApJ...723.1255R", "2006ApJ...639L..51A", "2001AJ....122.2993S", "2010ASPC..423..355R", "2010A&A...518A..59G", "2010ApJ...721L..48K"], "name": "New difference library", "description": "Sample difference library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   466  100   407  100    59    407     59  0:00:01 --:--:--  0:00:01  2858


### /documents/&lt;library_id&gt;
Interact with a specific library. The `library_id` (string) is the unique identifier of the library, as specified in the `/libraries` GET response.

Functions allowed: add records to a library (POST), delete records from a library (POST), update the metadata of a library (PUT), delete the entire library (DELETE)

#### POST
Add or delete a document to a given library. Returns the number of documents added or removed

Specify that the request is a POST method by adding `-X POST` to your request. You must include a payload, passed via the `-d` flag, in JSON format. Add this header to your request to indicate your payload is in the appropriate format: `-H "Content-Type: application/json"`. The payload must include:
  * bibcode: (list) list of bibcodes to be added or removed
  * action: (string) 'add' or 'remove' to add or remove the given bibcodes from the specified library

*Return data*:
  * number_added: (integer) number of documents added (if 'add' is used)
  * number_removed: (integer) number of documents removed (if 'remove' is used)

*Permissions*:
The following type of user can add or remove documents:
  * owner
  * admin
  * write

In [9]:
%%bash
token="your-token-here"
# add a bibcode
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/documents/hHGU1Ef-TpacAhicI3J8kQ \
    -X POST \
    -d '{"bibcode": ["1974Natur.248...30H"], "action": "add"}'

{"number_added": 1}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    73  100    20  100    53     20     53  0:00:01 --:--:--  0:00:01   244


In [10]:
%%bash
token="your-token-here"
# remove it again
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/documents/hHGU1Ef-TpacAhicI3J8kQ \
    -X POST \
    -d '{"bibcode": ["1974Natur.248...30H"], "action": "remove"}'

{"number_removed": 1}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    22  100    56     22     56  0:00:01 --:--:--  0:00:01   295


#### PUT
Update the metadata of a given library. Returns updated metadata of the library

As with the POST method, you must specify that the request is a PUT method by adding `-X PUT` to your request. You must also include a payload, passed via the `-d` flag, in JSON format. Add this header to your request to indicate your payload is in the appropriate format: `-H "Content-Type: application/json"`. The payload can include:

*Payload*:
  * name: (string) name of the library
  * description: (string) description of the library
  * public: (boolean) true if the library should be publicly viewable, false if it should be private (note that this is case sensitive and should be lower case)

*Note: The above are optional. Leave fields out if they do not need to be updated.*

*Return data*:
  * name: (string) name of the library
  * description: (string) description of the library
  * public:(boolean) true if the library is publicly viewable, false if it is private

*Note: returns the key/value that was requested to be updated*

*Permissions*:
The following type of user can update the 'name', 'library', and 'public' attributes:
  - owner
  - admin

In [11]:
%%bash
token="your-token-here"
# change the metadata of a library
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/documents/hHGU1Ef-TpacAhicI3J8kQ \
    -X PUT \
    -d '{"name": "New API library name", "description": "New API library description", "public": false}'

{"description": "New API library description", "name": "New API library name", "public": false}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   191  100    96  100    95     96     95  0:00:01 --:--:--  0:00:01   313


##### DELETE
Delete the given library

Make sure to specify the DELETE method using `-X DELETE`. No payload is required, as the library ID is passed in the URL. There is no return response to the request.

*Permissions*: The following type of user can update a library:
  - owner

In [12]:
%%bash
token="your-token-here"
# create a temporary library
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/libraries \
    -X POST \
    -d '{"name":"temp library"}'


{"id": "GPdVMFKwTmC87ysCk3JK3g", "description": "My ADS library", "name": "temp library"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   113  100    90  100    23     90     23  0:00:01 --:--:--  0:00:01   379


In [13]:
%%bash
token="your-token-here"
# delete the temporary library
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/documents/GPdVMFKwTmC87ysCk3JK3g \
    -X DELETE

{}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0      3      0  0:00:01 --:--:--  0:00:01    10


### /permissions/&lt;library_id&gt;
Edit the permissions of a library. The `library_id` (string) is the unique identifier of the library, as specified in the `/libraries` GET response.

#### GET
Get the permissions for a given library.

The response is a list of dictionaries, where each dictionary is for a specific user. The dictionary contains a list of all the permissions of the user:
  	* user dictionary (dict) contains:
  		* key (string): user e-mail
  		* item (list): list of permissions ('read', 'write', 'admin', 'owner')

*Permissions*:
The following type of user can access permissions of a library:
  - owner
  - admin

In [14]:
%%bash
token="your-token-here"
# get the library permissions
curl -H "Authorization: Bearer $token" \
    https://api.adsabs.harvard.edu/v1/biblib/permissions/hHGU1Ef-TpacAhicI3J8kQ
    

[{"test.user-api@example.email.com": ["owner"]}]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    52  100    52    0     0     52      0  0:00:01 --:--:--  0:00:01   201


#### POST
Edit the permissions of a library.

Specify that the request is a POST method by adding `-X POST` to your request. You must include a payload, passed via the `-d` flag, in JSON format. Add this header to your request to indicate your payload is in the appropriate format: `-H "Content-Type: application/json"`. The payload must include:
  * email: (string) specifies which user's permissions to be modified
  * permission: (dict) specifies the permissions type and whether the user has this permission
   - allowed keys: (string) 'read', 'write', or 'admin'
   - allowed values: (boolean) whether the user has this permission (true=yes, false=no)

There is no response to the request.

Note: An email will be sent to the affected user upon a successful permissions change.

*Permissions*:
The following type of user can update a permission for a user for a given library:
  - owner
  - admin

In [15]:
%%bash
token="your-token-here"
# edit the library permissions
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/permissions/hHGU1Ef-TpacAhicI3J8kQ \
    -X POST \
    -d '{"email":"test.user-api@example.email.com", "permission": {"read": true}}'

{}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    67  100     3  100    64      3     64  0:00:01 --:--:--  0:00:01   179


### /transfer/&lt;library_id&gt;
Transfer a the ownership of a library to another ADS user. The `library_id` (string) is the unique identifier of the library, as specified in the `/libraries` GET response.

#### POST
Transfer the ownership of a library from one user to another. 

Specify that the request is a POST method by adding `-X POST` to your request. You must include a payload, passed via the `-d` flag, in JSON format. Add this header to your request to indicate your payload is in the appropriate format: `-H "Content-Type: application/json"`. The payload must include:
  * email: (string) email of the user the account should be transfered to

There is no response to the request.

Note: An email will be sent to the affected user upon a successful permissions change.

*Permissions*:
The following type of user can transfer libraries:
  - owner

In [16]:
%%bash
token="your-token-here"
# transfer the library ownership
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/biblib/transfer/hHGU1Ef-TpacAhicI3J8kQ \
    -X POST \
    -d '{"email": "test.user@example.email.com"}'

{}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    46  100     3  100    43      3     43  0:00:01 --:--:--  0:00:01   133
